In [6]:
import torch
from torch.autograd import Variable
from torch.autograd import Function
from torchvision import models
from torchvision import utils
import cv2
import numpy as np
import json

In [4]:
cuda = torch.cuda.is_available()
print(cuda)

False


In [8]:
class_index = json.load(open('data/imagenet_class_index.json', 'r'))

In [12]:
model = models.vgg19(pretrained=True)

In [13]:
model

VGG(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)
    (18): MaxPool

In [52]:
class FeatureExtractor():
    def __init__(self, model, target_layers):
        self.model = model
        self.target_layers = target_layers
        self.gradients = []
    
    def save_gradient(self, grad):
        self.gradients.append(grad)
    
    def __call__(self, x):
        outputs = []
        self.gradients = []
        # model.featuresに含まれる各レイヤに順に通す
        for name, module in self.model._modules.items():
            x = module(x)
            if name in self.target_layers:
                # xの勾配が計算されるときに呼び出せる関数を指定する
                # 勾配情報を保存する
                x.register_hook(self.save_gradient)
                # target_layersで指定された層の出力を保存する
                outputs += [x]
        return outputs, x

In [ ]:
class ModelOutput():
    """
    forward pass
    1. ネットワークの出力
    2. 対象レイヤのactivations
    3. 対象レイヤの勾配
    を返す
    """
    def __init__(self, model, target_layers):
        self.model = model
        self.feature_extractor = FeatureExtractor(self.model.features, target_layers)
    
    def get_gradients(self):
        return self.feature_extractor.gradients
    
    def __call__(self, x):
        # target_activations = target_layersで指定した層の出力
        # output = ネットワークの (features) の出力
        target_activations, output = self.feature_extractor(x)
        
        # classiferの出力を計算
        output = output.view(output.size(0), -1)
        output = self.model.classifier(output)
        return target_activations, output

In [53]:
class GradCam:
    def __init__(self, model, target_layer_names, use_cuda):
        self.model = model
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = self.model.cuda()
        self.extractor = ModelOutputs(self.model, target_layer_names)
    
    def forward(self, input):
        return self.model(input)
    
    def __call__(self, input, index=None):
        # features = target_layer_namesで指定した層の出力
        # output = モデルの分類層の出力
        if self.cuda:
            features, output = self.extractor(input.cuda())
        else:
            features, output = self.extractor(input)
        print(features.size(), output.size())

In [54]:
x = Variable(torch.rand(1, 3, 224, 224), requires_grad=True)

In [61]:
gradients = []
def save_gradient(grad):
    gradients.append(grad)

v = Variable(torch.Tensor([0, 0, 0]), requires_grad=True)
h = v.register_hook(save_gradient)
v.backward(torch.Tensor([10, 10, 10]))
print(v.grad.data)
print(gradients)


 10
 10
 10
[torch.FloatTensor of size 3]

[Variable containing:
 10
 10
 10
[torch.FloatTensor of size 3]
]
